# Charlando con mi CLI vía Cisco pyATS 🚀
---

🔁 Antes de comenzar, es necesario crear un entorno virtual con los siguientes comandos:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

### 1️⃣ Creando mi archivo de topología
Ahora, toca crear el archivo `topology.yaml` donde especificaremos cuáles son nuestros dispositivos, y cómo podemos conectarnos a ellos.</br>

### 2️⃣ Leyendo mis configs
Excelente. Ahora es momento de ponerlos el sombrero de programadores y escribir algo de código.</br>
Vamos a conectarnos a nuestro dispositivo mencionado en el archivo `testbed.yaml`.</br>
pyATS usará la biblioteca `unicon` y la información en el archivo yaml para intentar crear una conexión SSH con el dispositivo `sandbox1`.

In [1]:
import json
from pyats.topology import loader

In [ ]:
testbed = loader.load('pyATS/testbed.yaml')
device = testbed.devices['sandbox1']

device.connect()
print("✅ Connected successfully to", device.name)

🔌 Excelso. Lo que queremos ahora es saber **cuáles son las interfaces de este dispositivo**.</br>
**Usemos [los parsers Genie que vienen incluídos 🧞](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers)** para conseguir esta información en un JSON.

In [ ]:
genie_parsed_interfaces = device.parse("show ip interface brief")
print(f"\n\n\n\n\n{json.dumps(genie_parsed_interfaces,indent=4)}")

### 3️⃣ Hacer y deshacer una interfaz
🤖 Ahora usemos [las clases de pyATS](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/models) para crear una interfaz Loopback. pyATS se encargará de todos los pasos intermedios para crear la interfaz. Nosotros solamente tenemos qué proveer la información mínima necesaria.

In [ ]:
from genie.conf.base import Interface

new_iosxr_interface = Interface(name="Loopback505", device=device)
new_iosxr_interface.description = "TechTilesDemo"
new_iosxr_interface.build_config()

🔥 Finalmente, basta solo cambiar una función para deshacer esta configuración.

In [ ]:
iface = device.interfaces.get("Loopback505")
rollback_config = iface.build_unconfig()
print(rollback_config)